### 딥러닝

층을 깊게 한 심층 신경망

### 더 깊은 신경망

ex)

<img src='./img/deeplearning_1.png' width=500>

In [1]:
import pickle
import numpy as np
from collections import OrderedDict
from common.layers import *


class DeepConvNet:
    """정확도 99% 이상의 고정밀 합성곱 신경망

    네트워크 구성은 아래와 같음
        conv - relu - conv- relu - pool -
        conv - relu - conv- relu - pool -
        conv - relu - conv- relu - pool -
        affine - relu - dropout - affine - dropout - softmax
    """
    def __init__(self, input_dim=(1, 28, 28),
                 conv_param_1 = {'filter_num':16, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_2 = {'filter_num':16, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_3 = {'filter_num':32, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_4 = {'filter_num':32, 'filter_size':3, 'pad':2, 'stride':1},
                 conv_param_5 = {'filter_num':64, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_6 = {'filter_num':64, 'filter_size':3, 'pad':1, 'stride':1},
                 hidden_size=50, output_size=10):
        # 가중치 초기화
        # 각 층의 뉴런 하나당 앞 층의 몇 개 뉴런과 연결되는가
        pre_node_nums = np.array([1*3*3, 16*3*3, 16*3*3, 32*3*3, 32*3*3, 64*3*3, 64*4*4, hidden_size])
        wight_init_scales = np.sqrt(2.0 / pre_node_nums)  # ReLU를 사용할 때의 권장 초깃값
        
        self.params = {}
        pre_channel_num = input_dim[0]
        for idx, conv_param in enumerate([conv_param_1, conv_param_2, conv_param_3, conv_param_4, conv_param_5, conv_param_6]):
            self.params['W' + str(idx+1)] = wight_init_scales[idx] * np.random.randn(conv_param['filter_num'], pre_channel_num, conv_param['filter_size'], conv_param['filter_size'])
            self.params['b' + str(idx+1)] = np.zeros(conv_param['filter_num'])
            pre_channel_num = conv_param['filter_num']
        self.params['W7'] = wight_init_scales[6] * np.random.randn(64*4*4, hidden_size)
        self.params['b7'] = np.zeros(hidden_size)
        self.params['W8'] = wight_init_scales[7] * np.random.randn(hidden_size, output_size)
        self.params['b8'] = np.zeros(output_size)

        # 계층 생성
        self.layers = []
        self.layers.append(Convolution(self.params['W1'], self.params['b1'], 
                           conv_param_1['stride'], conv_param_1['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W2'], self.params['b2'], 
                           conv_param_2['stride'], conv_param_2['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Convolution(self.params['W3'], self.params['b3'], 
                           conv_param_3['stride'], conv_param_3['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W4'], self.params['b4'],
                           conv_param_4['stride'], conv_param_4['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Convolution(self.params['W5'], self.params['b5'],
                           conv_param_5['stride'], conv_param_5['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W6'], self.params['b6'],
                           conv_param_6['stride'], conv_param_6['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Affine(self.params['W7'], self.params['b7']))
        self.layers.append(Relu())
        self.layers.append(Dropout(0.5))
        self.layers.append(Affine(self.params['W8'], self.params['b8']))
        self.layers.append(Dropout(0.5))
        
        self.last_layer = SoftmaxWithLoss()

    def predict(self, x, train_flg=False):
        for layer in self.layers:
            if isinstance(layer, Dropout):
                x = layer.forward(x, train_flg)
            else:
                x = layer.forward(x)
        return x

    def loss(self, x, t):
        y = self.predict(x, train_flg=True)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)

        acc = 0.0

        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx, train_flg=False)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt)

        return acc / x.shape[0]

    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        tmp_layers = self.layers.copy()
        tmp_layers.reverse()
        for layer in tmp_layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        for i, layer_idx in enumerate((0, 2, 5, 7, 10, 12, 15, 18)):
            grads['W' + str(i+1)] = self.layers[layer_idx].dW
            grads['b' + str(i+1)] = self.layers[layer_idx].db

        return grads

    def save_params(self, file_name="params.pkl"):
        params = {}
        for key, val in self.params.items():
            params[key] = val
        with open(file_name, 'wb') as f:
            pickle.dump(params, f)

    def load_params(self, file_name="params.pkl"):
        with open(file_name, 'rb') as f:
            params = pickle.load(f)
        for key, val in params.items():
            self.params[key] = val

        for i, layer_idx in enumerate((0, 2, 5, 7, 10, 12, 15, 18)):
            self.layers[layer_idx].W = self.params['W' + str(i+1)]
            self.layers[layer_idx].b = self.params['b' + str(i+1)]

위 신경망의 특징  
  
- $3\times 3$의 작은 필터를 사용한 합성곱 계층
- 활성화 함수는 ReLU
- 완전연결 계층 뒤에 드롭아웃 계층 사용
- Adam을 사용하여 최적화
- 가중치 초깃값은 'He 초깃값'
- 층이 깊어지면서 채널 수가 늘어남

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from dataset.mnist import load_mnist
from common.trainer import Trainer

(x_train, t_train), (x_test, t_test) = load_mnist(flatten=False)

network = DeepConvNet()  
trainer = Trainer(network, x_train, t_train, x_test, t_test,
                  epochs=20, mini_batch_size=100,
                  optimizer='Adam', optimizer_param={'lr':0.001},
                  evaluate_sample_num_per_epoch=1000)
trainer.train()

# 매개변수 보관
network.save_params("deep_convnet_params.pkl")
print("Saved Network Parameters!")

### 데이터 확장

데이터 확장(data augmentation) : 입력 이미지(훈련 이미지)를 알고리즘을 통해 '인위적'으로 확장  
입력 이미지를 회전, 이동, 일부를 잘라냄(crop), 좌우 반전(flip) 등의 다양한 방법으로 이미지를 확장함  
훈련 이미지의 개수를 늘리면 딥러닝의 인식 수준을 개선할 수 있음

### 깊게 하는 이유

딥러닝에서 층을 깊게 할 때의 이점  
- 신경망의 매개변수 수가 줄어듬  
    층을 깊게 한 신경망은 깊지 않은 경우보다 적은 매개변수로 같은 (혹은 그 이상) 수준의 표현력을 달성할 수 있음

ex) $5\times 5$ 필터로 구성된 합성곱 계층

<img src='./img/deeplearning_2.png' width=500>

$3\times 3$ 필터로 합성곱 연산을 2회 반복

<img src='./img/deeplearning_3.png' width=500>

$5\times 5$ 합성곱 연산 1회는 매개변수의 수가 25개($5\times 5$)인 반면,  
$3\times 3$ 합성곱 연산 2회는 매개변수의 수가 18개($2\times 3\times 3$)으로,  
매개변수의 수는 층을 반복할수록 적어짐  
또한 그 개수의 차이는 층이 깊어질수록 커짐

또한, 작은 필터를 겹쳐 신경망을 깊게 할 때의 장점은 매개변수의 수를 줄여 넓은 수용 영역을 소화할 수 있음  
게다가 층을 거듭하면서 ReLU 등의 활성화 함수를 합성곱 계층 사이에 끼움으로써 신경망의 표현력이 개선됨  
활성화 함수가 신경망에 '비선형' 힘을 가하고, 비선형 함수가 겹치면서 더 복잡한 것도 표현할 수 있게 되기 때문

- 학습의 효율성  
    층을 깊게 함으로써 학습 데이터의 양을 줄여 학습을 고속으로 수행할 수 있음

- 학습해야 할 문제를 계층적으로 분해할 수 있음  
    각 층이 학습해야 할 문제를 더 단순한 문제로 대체할 수 있음

- 정보를 계층적으로 전달할 수 있음  
    예를 들어 에지를 추출한 층의 다음 층은 에지 정보를 쓸 수 있고, 더 고도의 패턴을 효과적으로 학습할 수 있음

### VGG

합성곱 계층과 풀링 계층으로 구성되는 '기본적'인 CNN  
비중 있는 층(합성곱 계층, 완전연결 계층)을 모두 16층(혹은 19층)으로 심화  
(층의 깊이에 따라서 'VGG16'과 'VGG19'로 구분하기도 함)  

<img src='./img/VGG.png' width=500>

$3\times 3$의 작은 필터를 사용한 합성곱 계층을 연속적으로 거침  
합성곱 계층을 2~4회 연속으로 풀링 계층을 두어 크기를 절반으로 줄이는 처리를 반복하고  
마지막에는 완전연결 계층을 통과시켜 결과를 출력  
VGG는 구성이 간단하여 응용하기 좋음

### GoogLeNet

GoogLeNet은 가로 방향에 '폭'이 있음  
이를 '인셉션 구조'라 함

<img src='./img/GoogLeNet.png' width=500>

인셉션 구조는 위 그림과 같이 크기가 다른 필터(와 풀링)을 여러 개 적용하여 그 결과를 결합함  
이 인셉션 구조를 하나의 빌딩 블록(구성요소)로 사용하는 것이 GoogLeNet의 특징

### ResNet(Residual Network)

마이크로소프트의 팀이 개발한 네트워크  
  
딥러닝의 학습에서는 층이 지나치게 깊으면 학습이 잘 되지 않고, 오히려 성능이 떨어지는 경우도 많음  
ResNet에서는 스킵 연결(skip connection)을 도입하여 층의 깊이에 비례해 성능을 향상할 수 있게 함  

<img src='./img/ResNet.png' width=500>

\* 여기서, weight layer는 합성곱 계층을 의미  
  
입력 x를 연속한 두 합성곱 계층을 건너뛰어 출력에 바로 연결함  
이 스킵 연결로 인해 $F(x)+x$가 됨  
이는 역전파 때 신호감쇠를 막아주는 역할을 하여 층이 깊어져도 학습을 효율적으로 할 수 있도록 해줌  
ResNet은 VGG 신경망을 기반으로 스킵 연결을 도입하여 층을 깊이 한 모델

### 전이학습(transfer learning)

학습된 가중치(혹은 그 일부)를 다른 신경망에 복사한 다음,  
새로운 데이터셋을 대상으로 재학습(fine tuning)을 수행하는 방법  
전이 학습은 보유한 데이터셋이 적을 때 유용

### 딥러닝의 활용

### 사물 검출

사물검출 : 이미지 속에 담긴 사물의 위치와 종류(클래스)를 알아내는 기술  

- R-CNN(Regions with Convolutional Neural Network)

<img src='./img/R_CNN.png' width=500>  
  
사물에 위치한 영역을 찾아내고, 추출한 각 영역에 CNN을 적용하여 클래스를 분류함

- Faster R-CNN  
  
후보 영역 추출까지 CNN으로 처리하는 기법  
모든 일을 하나의 CNN에서 처리하기 때문에 빠름

### 분할

분할 : 이미지를 픽셀 수준에서 분류하는 문제

- FCN(Fully Convolutional Network)  

<img src='./img/FCN.png' width=500>  
  
CNN은 완전연결 계층을 이용하는 반면, FCN은 완전연결 계층을 '같은 기능을 하는 합성곱 계층'으로 바꿈  
공간 볼륨을 유지한 채 마지막 출력까지 처리  
단 한 번의 forward 처리로 모든 픽셀의 클래스를 분류  
  
마지막에 공간 크기를 확대하는 처리를 도입  
확대는 이중 선형 보간에 의한 선형 확대  
FCN에서는 이 선형 확대를 역합성곱 연산으로 구현함

### 사진 캡션 생성

사진 캡션 생성 : 사진을 주면, 그 사진을 설명하는 글을 자동으로 생성

- NIC(Neural Image Caption)

심층 CNN과 자연어를 다루는 순환 신경망(Recurrent Neural Network, RNN)으로 구성  
\* RNN은 순환적 관계를 갖는 신경망으로 자연어나 시계열 데이터 등의 연속된 데이터를 다룰 때 많이 활용됨  
  
NIC는 CNN으로 사진에서 특징을 추출하고, 그 특징을 RNN에 넘김  
RNN은 CNN이 추출한 특징을 초깃값으로 해서 텍스트를 '순환적'으로 생성함  
  
\* 이처럼 사진이나 자연어와 같은 여러 종류의 정보를 조합하고 처리하는 것을 멀티모달 처리(multimodal processing)이라고 함

### 이미지 생성

- GAN(Generative Adversarial Network)  

생성자(Generator)와 식별자(Discriminator)로 불리는 2개의 신경망을 이용  
  
생성자는 진짜와 똑같은 이미지를 생성하고,  
식별자는 그것이 진짜인지(생성자가 생성한 이미지인지, 아니면 실제로 촬영된 이미지인지)를 판정  
생성자와 식별자가 겨루도록 학습시켜, 생성자는 더 정교한 가짜 이미지 생성 기술을 학습하고 식별자는 더 정확하게 간파할 수 있는 감정사로 성장

### Deep Q-Network(DQN, 강화학습)

강화학습(reinforcement learning) : '가르침'에 의존하는 '지도 학습'과는 다른 분야로, 시행착오 과정에서 스스로 학습하게 하는 분야

<img src='./img/Deep Q-Network.png' width=200>

강화학습에서는 에이전트가 환경에 맞게 행동을 선택하고,  
그 행도엥 의해서 환경이 변한다는 것이 기본적인 틀  
환경이 변화하면 에이전트는 어떠한 보상을 얻으며,  
강화학습의 목적은 더 나은 보상을 받는 쪽으로 에이전트의 행동 지침을 바로잡는 것